# 데이터 진단 및 문제 파악

분석 결과에서 "정의되지 않은 코드" 문제를 파악하고 해결합니다.

In [1]:
import sys
import os

# 프로젝트 루트를 Python 경로에 추가
project_root = os.path.abspath('../../..')
if project_root not in sys.path:
    sys.path.insert(0, project_root)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Variable Decoder 최신 버전 import (fallback 지원)
from notebooks.vfxpedia.utils.variable_decoder import (
    VariableDecoder,
    get_korean_label,
    get_korean_labels
)

# 팀 시각화 스타일 설정
import matplotlib.font_manager as fm

# 폰트 설정 (HMFMMUEX.TTC 우선, 없으면 Malgun Gothic)
try:
    font_path = "C:/Windows/Fonts/HMFMMUEX.TTC"
    if os.path.exists(font_path):
        font_prop = fm.FontProperties(fname=font_path)
        plt.rcParams['font.family'] = font_prop.get_name()
    else:
        plt.rcParams['font.family'] = 'Malgun Gothic'
except:
    plt.rcParams['font.family'] = 'Malgun Gothic'

plt.rcParams['axes.unicode_minus'] = False

# 팀 컬러 정의
TEAM_COLORS = {
    'primary': '#1f77b4',    # 파란색
    'success': '#ff7f0e',    # 주황색 (churn=1)
    'danger': '#1f77b4',     # 파란색 (churn=0)
    'palette': ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']
}

# 데이터 로드
df = pd.read_csv('../../../data/analy_data.csv')
print(f"데이터 shape: {df.shape}")

# VariableDecoder 초기화 (fallback 지원)
decoder = VariableDecoder()
print('✅ VariableDecoder 초기화 완료!')

데이터 shape: (89822, 210)
✅ variable.csv 로드 완료: 702 개 매핑
✅ variable_full.csv 로드 완료: 681 개 매핑 (fallback)
✅ VariableDecoder 초기화 완료!


## 1. 각 변수의 실제 코드값 확인

In [2]:
# 분석 대상 변수
analysis_vars = ['sob_01z1', 'sob_02z1', 'soa_01z1', 'soa_06z2', 'soa_07z1', 'sod_02z3']

print("="*80)
print("📊 각 변수의 실제 코드값 분포")
print("="*80)

undefined_issues = []  # 문제 수집

for var in analysis_vars:
    var_label = decoder.get_variable_label(var)
    print(f"\n### {var} - {var_label}")
    print("-" * 60)
    
    # 실제 데이터의 코드값
    actual_values = df[var].value_counts().sort_index()
    print("\n실제 데이터의 코드값:")
    print(actual_values)
    
    # VariableDecoder에 정의된 코드값
    code_mapping = decoder.get_code_mapping(var)
    if code_mapping:
        defined_codes = set(code_mapping.keys())
        print(f"\nVariableDecoder에 정의된 코드: {sorted(defined_codes)}")
        
        # 정의되지 않은 코드 찾기
        actual_codes = set(actual_values.index.dropna())
        undefined_codes = actual_codes - defined_codes
        
        if undefined_codes:
            print(f"\n⚠️ 정의되지 않은 코드: {sorted(undefined_codes)}")
            print(f"   해당 코드의 빈도:")
            for code in sorted(undefined_codes):
                count = actual_values.get(code, 0)
                pct = count / len(df) * 100
                print(f"     {code}: {count}건 ({pct:.2f}%)")
                undefined_issues.append({
                    '변수': var,
                    '변수명': var_label,
                    '코드': code,
                    '건수': count,
                    '비율(%)': f"{pct:.2f}"
                })
        else:
            print("\n✅ 모든 코드가 정의되어 있음")
    else:
        print("\n⚠️ VariableDecoder에 매핑 정보 없음")
    
    print()

📊 각 변수의 실제 코드값 분포

### sob_01z1 - 교육수준
------------------------------------------------------------

실제 데이터의 코드값:
sob_01z1
1.0      1465
2.0        54
3.0      9897
4.0     10378
5.0     31369
6.0     11171
7.0     21721
8.0      3733
77.0       33
99.0        1
Name: count, dtype: int64

VariableDecoder에 정의된 코드: [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 77.0, 99.0]

✅ 모든 코드가 정의되어 있음


### sob_02z1 - 졸업여부
------------------------------------------------------------

실제 데이터의 코드값:
sob_02z1
1.0    78671
2.0      346
3.0     7377
4.0     1875
7.0       33
8.0     1519
9.0        1
Name: count, dtype: int64

VariableDecoder에 정의된 코드: [1.0, 2.0, 3.0, 4.0, 7.0, 8.0, 9.0]

✅ 모든 코드가 정의되어 있음


### soa_01z1 - 경제활동여부
------------------------------------------------------------

실제 데이터의 코드값:
soa_01z1
1.0    64823
2.0    24995
7.0        1
9.0        3
Name: count, dtype: int64

VariableDecoder에 정의된 코드: [1.0, 2.0, 7.0, 9.0]

✅ 모든 코드가 정의되어 있음


### soa_06z2 - 직업분류
--------------------------------------

## 2. churn 변수 확인

In [3]:
print("\n📊 타겟 변수 (churn) 분석")
print("="*60)
print("\n분포:")
print(df['churn'].value_counts())
print(f"\n결측값: {df['churn'].isna().sum()}건")
print(f"금연 성공률: {df['churn'].mean() * 100:.2f}%")

# churn이 있는 데이터만 필터링
df_with_churn = df[df['churn'].notna()]
print(f"\nchurn이 있는 데이터: {len(df_with_churn)}건 ({len(df_with_churn)/len(df)*100:.1f}%)")


📊 타겟 변수 (churn) 분석

분포:
churn
1    49251
0    40571
Name: count, dtype: int64

결측값: 0건
금연 성공률: 54.83%

churn이 있는 데이터: 89822건 (100.0%)


## 3. 변수별 결측값 및 특수코드 분석

In [4]:
print("\n📊 변수별 데이터 품질 분석")
print("="*80)

quality_report = []

for var in analysis_vars:
    var_label = decoder.get_variable_label(var)
    
    # 결측값
    na_count = df[var].isna().sum()
    na_pct = na_count / len(df) * 100
    
    # 특수코드 (응답거부, 모름, 비해당 등)
    special_codes = []
    special_count = 0
    
    code_mapping = decoder.get_code_mapping(var)
    if code_mapping:
        for code, label in code_mapping.items():
            if any(keyword in str(label) for keyword in ['응답거부', '모름', '비해당']):
                special_codes.append(code)
                special_count += (df[var] == code).sum()
    
    special_pct = special_count / len(df) * 100
    
    # 유효 데이터
    valid_count = len(df) - na_count - special_count
    valid_pct = valid_count / len(df) * 100
    
    quality_report.append({
        '변수': var_label,
        '전체': len(df),
        '결측값': f"{na_count} ({na_pct:.1f}%)",
        '특수코드': f"{special_count} ({special_pct:.1f}%)",
        '유효데이터': f"{valid_count} ({valid_pct:.1f}%)"
    })

quality_df = pd.DataFrame(quality_report)
print(quality_df.to_string(index=False))


📊 변수별 데이터 품질 분석
      변수    전체      결측값          특수코드          유효데이터
    교육수준 89822 0 (0.0%)     34 (0.0%) 89788 (100.0%)
    졸업여부 89822 0 (0.0%)   1553 (1.7%)  88269 (98.3%)
  경제활동여부 89822 0 (0.0%)      4 (0.0%) 89818 (100.0%)
    직업분류 89822 0 (0.0%) 25008 (27.8%)  64814 (72.2%)
   종사상지위 89822 0 (0.0%) 25008 (27.8%)  64814 (72.2%)
sod_02z3 89822 0 (0.0%)      0 (0.0%) 89822 (100.0%)


## 4. 교차 분석: churn과 함께 유효한 데이터

In [5]:
print("\n📊 churn과 함께 분석 가능한 데이터")
print("="*80)

churn_quality = []

for var in analysis_vars:
    var_label = decoder.get_variable_label(var)
    
    # churn과 해당 변수 모두 유효한 데이터
    both_valid = df[(df['churn'].notna()) & (df[var].notna())]
    
    # 특수코드 제외
    code_mapping = decoder.get_code_mapping(var)
    if code_mapping:
        special_codes = [
            code for code, label in code_mapping.items()
            if any(keyword in str(label) for keyword in ['응답거부', '모름', '비해당'])
        ]
        both_valid_clean = both_valid[~both_valid[var].isin(special_codes)]
    else:
        both_valid_clean = both_valid
    
    churn_quality.append({
        '변수': var_label,
        '원본 데이터': len(df),
        '분석 가능': len(both_valid_clean),
        '비율(%)': f"{len(both_valid_clean)/len(df)*100:.1f}",
        '금연성공률(%)': f"{both_valid_clean['churn'].mean()*100:.1f}"
    })

churn_quality_df = pd.DataFrame(churn_quality)
print(churn_quality_df.to_string(index=False))


📊 churn과 함께 분석 가능한 데이터
      변수  원본 데이터  분석 가능 비율(%) 금연성공률(%)
    교육수준   89822  89788 100.0     54.8
    졸업여부   89822  88269  98.3     54.6
  경제활동여부   89822  89818 100.0     54.8
    직업분류   89822  64814  72.2     51.0
   종사상지위   89822  64814  72.2     51.0
sod_02z3   89822  89822 100.0     54.8


## 5. 진단 결과 요약

In [6]:
print("\n" + "="*80)
print("📋 진단 결과 요약 및 권장 조치")
print("="*80)

print("\n### 1️⃣ 정의되지 않은 코드값")
if undefined_issues:
    print(f"\n⚠️ 총 {len(undefined_issues)}개의 정의되지 않은 코드 발견")
    undefined_df = pd.DataFrame(undefined_issues)
    print("\n" + undefined_df.to_string(index=False))
    print("\n→ 조치: var_mapping.py에 위 코드값 추가 필요")
else:
    print("\n✅ 모든 코드값이 정의되어 있음")

print("\n### 2️⃣ 데이터 정제 방안")
print("\n✅ 권장: 특수코드(응답거부, 모름, 비해당) 제거")
print("✅ 권장: 결측값 제거")
print("⚠️ 주의: 제거 후 표본 크기 충분성 확인 필요")

print("\n### 3️⃣ 분석 가능 표본 크기")
min_sample = churn_quality_df['분석 가능'].min()
print(f"\n최소 표본 크기: {min_sample:,}건")

# 숫자로 변환하여 비교
if min_sample > 1000:
    print("✅ 충분한 표본 크기 확보")
elif min_sample > 500:
    print("⚠️ 표본 크기가 다소 작음. 해석 시 주의 필요")
else:
    print("❌ 표본 크기가 매우 작음. 통계적 검정력 부족 우려")

print("\n### 4️⃣ 다음 단계")
print("\n1. variable.csv 업데이트 (정의되지 않은 코드 추가)")
print("2. 데이터 정제 스크립트 실행")
print("3. 분석 재실행")
print("4. 특성 중요도 분석 추가")
print("5. 최종 해석 및 보고서 작성")

print("\n" + "="*80)
print("✅ 진단 완료!")
print("="*80)


📋 진단 결과 요약 및 권장 조치

### 1️⃣ 정의되지 않은 코드값

✅ 모든 코드값이 정의되어 있음

### 2️⃣ 데이터 정제 방안

✅ 권장: 특수코드(응답거부, 모름, 비해당) 제거
✅ 권장: 결측값 제거
⚠️ 주의: 제거 후 표본 크기 충분성 확인 필요

### 3️⃣ 분석 가능 표본 크기

최소 표본 크기: 64,814건
✅ 충분한 표본 크기 확보

### 4️⃣ 다음 단계

1. variable.csv 업데이트 (정의되지 않은 코드 추가)
2. 데이터 정제 스크립트 실행
3. 분석 재실행
4. 특성 중요도 분석 추가
5. 최종 해석 및 보고서 작성

✅ 진단 완료!


## 6. 체크리스트 출력

In [7]:
print("\n📋 체크리스트\n")
print("="*80)

print(f"\n✅ 데이터 기본 정보:")
print(f"   - 전체 데이터: {len(df):,}건")
print(f"   - churn 있는 데이터: {len(df_with_churn):,}건")
print(f"   - 전체 금연 성공률: {df['churn'].mean()*100:.2f}%")

print(f"\n✅ 정의되지 않은 코드:")
if undefined_issues:
    print(f"   ⚠️ {len(undefined_issues)}개 발견")
    for issue in undefined_issues:
        print(f"   - {issue['변수명']}: 코드 {issue['코드']} ({issue['건수']:,}건, {issue['비율(%)']}%)")
else:
    print("   ✅ 없음")

print(f"\n✅ 분석 가능 표본:")
print(f"   - 최소: {churn_quality_df['분석 가능'].min():,}건")
print(f"   - 최대: {churn_quality_df['분석 가능'].max():,}건")

print(f"\n✅ 특이사항:")
# 표본이 가장 작은 변수
min_idx = churn_quality_df['분석 가능'].idxmin()
print(f"   - 표본이 가장 작은 변수: {churn_quality_df.loc[min_idx, '변수']}")

print("\n" + "="*80)
print("💡 이 정보를 최적화된 분석 노트북을 생성합니다!")
print("="*80)


📋 체크리스트


✅ 데이터 기본 정보:
   - 전체 데이터: 89,822건
   - churn 있는 데이터: 89,822건
   - 전체 금연 성공률: 54.83%

✅ 정의되지 않은 코드:
   ✅ 없음

✅ 분석 가능 표본:
   - 최소: 64,814건
   - 최대: 89,822건

✅ 특이사항:
   - 표본이 가장 작은 변수: 직업분류

💡 이 정보를 최적화된 분석 노트북을 생성합니다!
